<a href="https://colab.research.google.com/github/tintamaria95/CocoriGo/blob/main/importGolois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

--2022-11-16 18:52:57--  https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138784129 (132M) [application/zip]
Saving to: ‘project2022.zip’

project2022.zip     100%[===================>] 132.35M  35.6MB/s    in 4.3s    

2022-11-16 18:53:02 (30.8 MB/s) - ‘project2022.zip’ saved [138784129/138784129]

Archive:  project2022.zip
  inflating: Board.h                 
  inflating: Game.h                  
  inflating: Rzone.h                 
  inflating: compile.sh              
  inflating: compileMAC.sh           
  inflating: games.data              
  inflating: golois.cpp              
  inflating: golois.cpython-310-x86_64-linux-gnu.so  
  inflating: golois.cpython-37m-x86_64-linux-gnu.so  
  inflating: golois.cpython-38-x86_64-linux-gnu.so  
  inflat

In [ ]:
!python golois.py

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers
import gc

import golois

planes = 31
moves = 361
N = 10000
filters = 25

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')


print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

def block(input: keras.layers, nb_filters):
  x_1 = layers.Conv2D(filters, 3, strides=(1, 1), padding='same')(input)
  b_1 = layers.BatchNormalization()(x_1)
  r_1 = layers.ReLU()(b_1)
  x_2 = layers.Conv2D(filters, 5, strides=(1, 1), padding='same')(r_1)
  b_1_bis = layers.BatchNormalization()(x_2)

  i_1 = layers.Conv2D(filters, 1, strides=(1, 1), padding='same')(input)
  n_1 = layers.BatchNormalization()(i_1)

  z_1 = tf.keras.layers.Add()([b_1_bis, n_1])
  z_1_bis = layers.ReLU()(z_1)
  return z_1_bis

input_model = keras.Input(shape=(19, 19, planes), name='board')
x = block(input=input_model, nb_filters=filters)
for i in range(3):
  x = block(x, nb_filters=filters)
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
model = keras.Model(inputs=input_model, outputs=[policy_head, value_head])

model.summary ()

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.003, momentum=.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

#display(keras.utils.plot_model(model, str('test_0' + ".png"), show_shapes=True))
model.save ('test.h5')


##Réentraînement de modèle

In [ ]:
from tensorflow import keras
import numpy as np
from tqdm import tqdm
import gc

import golois

epochs = 200

planes = 31
moves = 361
N = 10000
batch = 128

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')
golois.getValidation (input_data, policy, value, end)

# Load models
model_name = "resnet_372.h5"
model = keras.models.load_model(model_name)
m_names = model.metrics_names

for i in tqdm(range(1, epochs + 1)):
    # print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, batch_size=batch, verbose=0)
    if (i % epochs == 0 or i % 10 == 0):
        gc.collect ()
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("policy_accuracy =", val[3])
        model.save (model_name)

print("")
print("*******************")
print(model_name)
for i, metric in enumerate(m_names):
  print(metric, ":", val[i])
#display(keras.utils.plot_model(model, str(model_name + ".png"), show_shapes=True))

  5%|▌         | 10/200 [00:43<15:47,  4.99s/it]

policy_accuracy = 0.3734000027179718


 10%|█         | 20/200 [01:21<13:30,  4.50s/it]

policy_accuracy = 0.3833000063896179


 15%|█▌        | 30/200 [02:01<13:39,  4.82s/it]

policy_accuracy = 0.3750999867916107


 20%|██        | 40/200 [02:36<10:54,  4.09s/it]

policy_accuracy = 0.3752000033855438


 20%|██        | 41/200 [02:40<10:08,  3.83s/it]

# Comparaison de modèles

In [ ]:
from tensorflow import keras
import numpy as np
import gc
import os

import golois

planes = 31
moves = 361
N = 10000
batch = 128

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')
#print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)
golois.getValidation (input_data, policy, value, end)

# Load models
h5_files = []
for name in os.listdir():
  if name[-2:] == 'h5':
    h5_files.append(name)

# if only one model
#h5_files = ['model_resnet_Adam_lr1_10-4.h5']
for model_filename in h5_files:
  model = keras.models.load_model(model_filename)
  m_names = model.metrics_names

  val = model.evaluate (input_data,
                        [policy, value], verbose = 0, batch_size=batch)
  print("")
  print("*******************")
  print(model_filename)
  print('Total params:', model.count_params())
  for i, metric in enumerate(m_names):
    print(metric, ":", val[i])
# display(keras.utils.plot_model(model, str(model_name + ".png"), show_shapes=True))


*******************
model_resnet_SGD_lr2_10-3.h5
Total params: 100071
loss : 3.483469247817993
policy_loss : 2.7814977169036865
value_loss : 0.693090558052063
policy_categorical_accuracy : 0.3434999883174896
value_mse : 0.12001872807741165

*******************
model_resnet_SGD_lr2_10-3_parall.h5
Total params: 98087
loss : 3.5874218940734863
policy_loss : 2.8910937309265137
value_loss : 0.6874495148658752
policy_categorical_accuracy : 0.3328000009059906
value_mse : 0.11721362173557281

*******************
model_resnet_Adam_lr1_10-4.h5
Total params: 99303
loss : 3.5075764656066895
policy_loss : 2.818538188934326
value_loss : 0.6879618167877197
policy_categorical_accuracy : 0.33309999108314514
value_mse : 0.11746852099895477

*******************
model_resnet_SGD_lr5_10-3.h5
Total params: 99303
loss : 3.3060922622680664
policy_loss : 2.6142449378967285
value_loss : 0.6866090297698975
policy_categorical_accuracy : 0.36559998989105225
value_mse : 0.11679057776927948
